# Assignment description

In this assignment, you will be performing more complex analysis using the full iowa dataset.   In contrast to previous assignments, where we directly wrote SQL queries using the magic `%%sql` cells, we will be directly connecting to the database and running queries through the Python database client.   This is how the magic `%%sql` cells are implemented under the covers anyways.

We will also be using [DuckDB](https://duckdb.org/), a new database system designed for analytics.  It is very similar to the SQLite database we have used in the past, however it is _must faster_ when analyzing the entire dataset.

# Setup code 
The following three blocks only need to run once.   It will install duckdb, download the data files, and load it into duckdb.



In [1]:
!pip install duckdb
!rm iowa.csv* iowa.duckdb*
!wget https://www.dropbox.com/s/0f4g8xa5m2s898i/iowa.csv

Traceback (most recent call last):
  File "/usr/local/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip._internal.cli.main'
fish: No matches for wildcard 'iowa.csv*'. See `help expand`.
rm iowa.csv* iowa.duckdb*
   ^
--2022-01-27 21:12:39--  https://www.dropbox.com/s/0f4g8xa5m2s898i/iowa.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0f4g8xa5m2s898i/iowa.csv [following]
--2022-01-27 21:12:39--  https://www.dropbox.com/s/raw/0f4g8xa5m2s898i/iowa.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9a6d7c54db13916438316c7b22.dl.dropboxusercontent.com/cd/0/inline/BekwgcqNm6cDqbsV4GgLnPB7pZsiREk47EbsmitVnoJhD9t5Vu62NIHu5G1vdcy8ssDPVToTfT_EYVNsvMhm0SYPeBjgWjF6KASHapA8XgOmIx-B7R1eUP

In [2]:
import duckdb
db = duckdb.connect('iowa.db')
db.execute("CREATE TABLE iowa AS SELECT * FROM read_csv_auto('iowa.csv')")

In [3]:
def runq(q):
  "I'm helper function to run and print queries"
  cursor = db.execute(q)
  df = cursor.fetchdf()
  print(df)
  return df


### **Q1.1** Which store had the most sales in terms of total records in the table?

In [ ]:
q11 = """
select store, count(1)
from iowa
group by store
order by count(1) desc
limit 1;
"""
runq(q11)

### Answer

store 2633

count 8934

### **Q1.2** At the store with the most total records (answer to Q1.1), what was the vendor number with most sale records? (The vendor that has the most records in the table?)

In [ ]:
q12 = """
select vendor_no, count(1)
from iowa
where store = 2633
group by vendor_no
order by count(1) desc
limit 1;
"""
runq(q12)

### Answer

vendor_no 260

count 1509

### **Q1.3**: For each zipcode, compute the single most purchased category_name by total `sale_bottles`.

a. We first need to get have a table with aggregated sale_bottles groupped by zipcode & category_name

In [ ]:
q13a = """CREATE OR REPLACE VIEW zip_cate_sum AS 
    SELECT zipcode, category_name, sum(sale_bottles) sum_qty
    FROM iowa
    GROUP BY zipcode, category_name
"""
runq(q13a)

b. With `zip_cate_sum` table, we can get the max `sum_qty` in a given zipcode with other groupby

In [ ]:
q13b = """CREATE OR REPLACE VIEW zip_cate_sum_max AS (
    SELECT zcs.zipcode, max(zcs.sum_qty) max_qty
    FROM zip_cate_sum zcs
    GROUP BY zcs.zipcode
)"""
runq(q13b)

c. use max bottle_qty sum and zipcode information, we can join above two tables to get category_name,
and use `ORDER BY` and `LIMIT` for expected output

In [ ]:
q13c = """
SELECT c.zipcode, c.category_name, c.sum_qty
FROM zip_cate_sum c, zip_cate_sum_max b
WHERE b.zipcode = c.zipcode AND b.max_qty = c.sum_qty
ORDER BY c.sum_qty desc
LIMIT 5;
"""
runq(q13c)

### The same as the above views, but as a single nested query

In [ ]:
q13nested = """
select c.zipcode, c.category_name, c.qty
from (
    select a.zipcode, max(a.qty) qty
    from (
        select zipcode, category_name, sum(sale_bottles) qty
        from iowa
        group by zipcode, category_name
    ) a
    group by a.zipcode
) b, 
(
    select zipcode, category_name, sum(sale_bottles) qty
    from iowa
    group by zipcode, category_name
) c
where b.zipcode = c.zipcode and b.qty = c.qty
order by c.qty desc
limit 5;
"""
runq(q13nested)


### Answer

```
(zipcode, category_name, qty)

0	52402	VODKA 80 PROOF	72031.0
1	52240	VODKA 80 PROOF	63878.0
2	50314	VODKA 80 PROOF	58215.0
3	50320	VODKA 80 PROOF	54927.0
4	50010	VODKA 80 PROOF	47651.0
```

### **Q1.4**: This problem has two steps, you only need to return the value from second step.

__Compute the set of all liquors with the characters "Lagavulin" (in lowercase) in its description (`im_desc`). The attribute `itemno` is the one that can serve as a unique identifier for a specific liquor.__

__Return the count of all distinct value of zipcode for all liquor stores that sold at least one of every type of Lagavulin as defined in the previous sentence.__

a. The number of different Lagavulins be made into one table

In [ ]:
q14a = """
CREATE OR REPLACE VIEW  item_ctn AS
    SELECT COUNT(DISTINCT(itemno)) as item_cnt
    FROM iowa
    WHERE im_desc LIKE '%Lagavulin%'
"""
db.execute(q14)

In [ ]:
runq("SELECT * FROM item_ctn limit 10;")

b. with selection of Lagavulin on description, and `group by` store and item, 
we can aggregate total sale_bottles, so that this `sum(sale_bottles)` is the __total sold bottles of one type of Lagavulin at one store__

In [ ]:
q14b = """CREATE OR REPLACE VIEW store_bottle_sum AS (
    SELECT store, itemno, sum(sale_bottles) qty_sum 
    FROM iowa
    WHERE im_desc LIKE '%Lagavulin%'
    GROUP BY store, itemno
) 
"""
runq(q14b)


c. We can select from `store_bottle_sum` table with qty_sum >= 2, to find out whether the sum bottle qty of a given type of Lagavulin in a given is qualified.

After this SELECTION, we can apply a `GROUP BY` on store to get the total qualfied number of Lagavulin of a given store.

In [ ]:
q14c = """CREATE OR REPLACE VIEW store_qualified_item AS (
    SELECT stbs.store, count(*) n_qualified
    FROM store_bottle_sum stbs
    WHERE stbs.qty_sum >= 1
    GROUP BY stbs.store
)"""
runq(q14c)


d. Join `store_qualified_tequila_item` and `tequila_item_ctn` to find out qualified store

In [ ]:
q14d = """CREATE OR REPLACE VIEW qualified_store AS (
    SELECT sqti.store 
    FROM store_qualified_item sqti, item_ctn tic
    WHERE sqti.n_qualified = tic.item_cnt
) 
"""
runq(q14d)

e. finally, join `qualified_store` on `iowa` to count distinct zipcode

In [79]:
q14e = """
SELECT count(distinct(i.zipcode))
FROM qualified_store qs, iowa i
WHERE qs.store = i.store;
"""
runq(q14e)

   count(i.zipcode)
0                 2


### A nested query version of the above



In [ ]:
q14nested = """
-- zip of all stores that have sold at least 2 of each Lagavolun, having Lagavulin in description

select count(distinct(e.zipcode))
from 
(
    select b.store, b.ctn
    from (
        select a.store, count(1) ctn
        from (
            select store, itemno, sum(sale_bottles) ctn
            from iowa
            where im_desc like '%Lagavulin%'
            group by store, itemno
        ) a
        where a.ctn >= 1
        group by a.store
    ) b, 
    (
        select count(distinct(itemno)) ctn
        from iowa
        where im_desc like '%Lagavulin%'
    ) c
    where b.ctn = c.ctn
) d
join iowa e
on e.store = d.store;
"""
runq(q14nested)

### Answer:

count: 2

### **Q1.5:** What is the item attribute of the liquor with the highest state bottle cost, (as defined below), across all of iowa?


First, let's try to compute the sum of the `state_bottle_retail` attribute.  What happens when you execute the following cell?

In [ ]:
runq("select sum(state_bottle_retail) from iowa")

Uh oh, the query throws an error!  To debug, let's take a look at the data:

In [ ]:
runq("select bottle_volume_ml from iowa")

Notice that the `state_bottle_retail` values have a `$` character!  That means the attribute is actually a `text` type, rather than a number!   

a. Write the correct query to sum the `state_bottle_retail` attribute.    You will want to write an expression that 1) removes the `$` character and 2) casts the string into a `numeric` type.

Take a look at the duckdb documentation for [text functions](https://duckdb.org/docs/sql/functions/char) and [casting data types](https://duckdb.org/docs/sql/expressions/cast)

In [ ]:
q15a = """
select sum(state_bottle_retail[1:]::numeric) as sum from iowa
"""
runq(q15a)

b. `GROUP BY` store and item to compute average state_bottle_retail per bottle_volume_ml for an item within store

In [ ]:
q15b = """CREATE OR REPLACE VIEW storewise_cost AS
SELECT store, itemno, avg(state_bottle_retail[1:]::numeric/bottle_volume_ml) as cost 
FROM iowa
WHERE bottle_volume_ml > 0
GROUP BY store, itemno;
"""
runq(q15b)

c. average across store for final result

In [ ]:
q15c = """
SELECT s.itemno, avg(s.cost) overall_cost
FROM storewise_cost s
GROUP BY itemno
ORDER BY avg(s.cost) DESC
LIMIT 1;
"""
runq(q15c)

### Nested query version of the above

In [ ]:
q15nested = """
select itemno, avg(per_store.cost) overall_cost
from (
    select store, itemno, avg(state_bottle_retail[1:]::numeric/bottle_volume_ml) as cost
    from iowa
    where bottle_volume_ml > 0
    group by store, itemno
) per_store
group by itemno
order by avg(per_store.cost) desc
limit 1;
"""
runq(q15nested)

### Answer

itemno: 927368

### Q1.6 (Answer not Limited to those)
`Python Pros:`

Python can perform more complicated logic on data, for example we can implement algorithms in an imperative 
programming language such as Python but not in SQL.

With python, we can achieve complex data visualizations over web applications, which cannot be done through SQL.

`Python Cons:`

Python syntax on data manipulate is obscure sometimes.

Python data manipulate is hard to recover and rollback.


`SQL Pros:`

SQL is specialized to data transformations.

SQL is safer and more efficient because DBMS knows how to analyze it.

SQL provides ACID properties for transactions in database.

`SQL Cons`:

Require cross-language data pipeline for brining SQL analysis into product.